In [1]:
import numpy as np
import pandas as pd
import os
import pickle

In [9]:
raw_data_dir = "../data/ft_data/"
new_dir = './RENET/'
FT_MODE = True

In [48]:
sentence_path = os.path.join(raw_data_dir, "sentences.txt")
ner_path = os.path.join(raw_data_dir, "anns.txt")
text_path = os.path.join(raw_data_dir, "docs.txt")



new_gene_file = open(new_dir+"genes_FINAL.befree", 'w')
new_disease_file = open(new_dir+"diseases_FINAL.befree", 'w')


_snt_file = open(new_dir+"sentences.txt", 'w')
_ann_file = open(new_dir+"anns.txt", 'w')
_doc_file = open(new_dir+"abstract.txt", 'w')


text_file = open(text_path, "r")
sentence_file = open(sentence_path, "r")
ner_file = open(ner_path, "r")

tar_pmid_lst = None

def get_sent_offset(sentences, text):
    sent_offset = []
    for i, sentence in enumerate(sentences):
        if i == 0:
            offset = 0
        else:
            offset = last_offset + len(last_sentence)
        if sentence != text[offset: offset+len(sentence)]:
            offset = offset + text[offset:].find(sentence)
        sent_offset.append(offset)        
        last_offset = offset
        last_sentence = sentence
    return sent_offset

def Sent_no(sent_offset, offset):
    return np.digitize(offset, sent_offset).tolist() - 1


def offset_in_sent(sentences_offset, sent_no, begin, end):
    begin_offset = begin - sentences_offset[sent_no]
    end_offset = end - sentences_offset[sent_no]
    return str(begin_offset) + "#" + str(end_offset)

def get_sentence(sentences, row):
    return sentences[sent_no]
    
    

_idx = 1
while (1):
    line = text_file.readline()
    if line == '':
        break
    pmid = line.strip()

    title = text_file.readline()
    title = title[:-1] if len(title) > 0 else title
    abstract = text_file.readline()
    abstract= abstract[:-1] if len(abstract) > 0 else abstract 
    para = text_file.readline()
    para = para[:-1] if len(para) > 0 else para 
    #fix space between title, abstract, para

    text = title + " " + abstract + " " + para

    text_file.readline()
    #text = abstract title + text

    sentences = []
    line = sentence_file.readline()
    if line == "\n":
        sentence_file.readline()
    while (1):
        line = sentence_file.readline()
        if line == "\n":
            break
        sentence = line.strip()
        sentences.append(sentence)

    sent_offset = get_sent_offset(sentences, text)

    snt_dict = {}
    anns = []
    while (1):
        line = ner_file.readline()
        if line == "\n":
            break
        ann = line.strip().split("\t")
        if ';' in ann[5]:
            ann[5] = ann[5].split(";")[1]
            
        # for ft only
        if FT_MODE:
            ann = ann[:-1]
#         print(ann)
        ann[1] = int(ann[1])
        ann[2] = int(ann[2])
        ann[-1], ann[-2] = ann[-2], ann[-1]
        if ann[4] == 'None' or len(ann[4]) < 1:
            continue
#         anns.append(ann[:])
#         print(ann)
        begin, end, mention, _id, _type = ann[1], ann[2], ann[3], ann[4], ann[5]
        
        if tar_pmid_lst and (pmid not in tar_pmid_lst):
            continue
#         print(_type)
        sent_no = Sent_no(sent_offset, begin)
        off_tar = offset_in_sent(sent_offset, sent_no, begin, end)
#         print(sent_no, off_tar)
        anns.append(ann[:] + [sent_no, off_tar])
        if sent_no in snt_dict:
            snt_dict[sent_no].append(ann[:] + [sent_no, off_tar])
        else:
            snt_dict[sent_no] = [ann[:] + [sent_no, off_tar]]
#     print(anns)
#     print(snt_dict)
    
    for _si in snt_dict:
        _info = [_i[5] for _i in snt_dict[_si]]
        if 'Gene' in _info and 'Disease' in _info:
            print(sentences[_si])
            print(snt_dict[_si])
            _snt_file.write('{}\n{}\n\n\n'.format(pmid, sentences[_si]))
            _doc_file.write('{}\n{}\n\n\n'.format(pmid, sentences[_si]))
            for _rec in snt_dict[_si]:
                _ann_file.write('{}\t{}\t{}\t{}\t{}\t{}\n'.format(_rec[0], _rec[-1].split('#')[0], _rec[-1].split('#')[1], _rec[3], _rec[5], _rec[4]))
            _ann_file.write('\n')
            
#     break
        
#         _pmid = pmid
#         _year, _journal_name, _journal_ISSN = '#', '#', '#'
#         _section, _section_nb, _sent_no = 'TITLE', 0, 00.8
        
#         if sent_no > 0:
#             _section, _section_nb, _sent_no = 'ALL_TEXT', 1, sent_no

#         _entity_id = _id
#         _entity_type = 'LONGTERM|DICTIONARY'
#         _entity_norm = mention.lower()
#         _mention = mention
#         _offset = off_tar
#         _entity_parent = 'nan'
#         _sentence = sentences[sent_no]
# #     'pubmed', 'year','journal_name', 
# # 'journal_ISSN', 'section', 'section_nb', 'sent_no', 'entity_id', 'entity_type', 'entity_norm', 'mention',  'offset',
# # 'entity_parent',  'sentence'

    
# #         print('%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t' % (\
# #                     _pmid, _year, _journal_name, _journal_ISSN, \
# #                     _section, _section_nb, _sent_no, \
# #                     _entity_id,  _entity_type,  _entity_norm, \
# #                     _mention, _offset, _entity_parent,\
# #                     _sentence))
# #         print(_type)
#         if _type == 'Disease':
#             new_disease_file.write('%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\n' % (\
#                     _pmid, _year, _journal_name, _journal_ISSN, \
#                     _section, _section_nb, _sent_no, \
#                     _entity_id,  _entity_type,  _entity_norm, \
#                     _mention, _offset, _entity_parent,\
#                     _sentence))
#         else:
#             new_gene_file.write('%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\n' % (\
#                     _pmid, _year, _journal_name, _journal_ISSN, \
#                     _section, _section_nb, _sent_no, \
#                     _entity_id,  _entity_type,  _entity_norm, \
#                     _mention, _offset, _entity_parent,\
#                     _sentence))
            
    if tar_pmid_lst and (pmid not in tar_pmid_lst):
#     if pmid in tar_pmid_lst:
        print(_idx, pmid)
        _idx += 1
#     print(abstract)
#     print(para)
    
#     print(sentences)
#     print(sent_offset)
    


#     print('\n'.join([str(i) for i in anns]))
#     break
    
print('end')
    
    
new_gene_file.close()
new_disease_file.close()
    

    
text_file.close()
sentence_file.close()
ner_file.close()

_snt_file.close()
_ann_file.close()
_doc_file.close()

Furthermore, at least some of these miRNAs are likely to be involved in tumorigenesis, as indicated in a recent study showing that the over-expression of the miR-17 cluster, when co-expressed with c-Myc, induces B cell lymphoma in mice.
[['16963499', 1979, 1985, 'miR-17', '723905', 'Gene', 12, '158#164'], ['16963499', 2040, 2048, 'lymphoma', 'D008223', 'Disease', 12, '219#227']]
Drosha functions in a complex known as Microprocessor that also contains a dsRNA-binding protein, DGCR8 (DiGeorge syndrome chromosomal region 8; also known as Pasha in Caenorhabditis elegans and Drosophila).
[['16963499', 2873, 2878, 'DGCR8', '54487', 'Gene', 17, '98#103'], ['16963499', 2934, 2939, 'Pasha', '54487', 'Gene', 17, '159#164'], ['16963499', 2775, 2781, 'Drosha', '29102', 'Gene', 17, '0#6'], ['16963499', 2880, 2897, 'DiGeorge syndrome', 'D004062', 'Disease', 17, '105#122']]
The dsRBD point mutants (mDRBD1, mDRBD2 and mDRBD1&2) had also lost RNA binding activity, as indicated by both UV-crosslinking a

Autosomal recessive Stargardt disease, the most common macular dystrophy, is caused by mutations in the gene encoding ABCA4, a photoreceptor ATP binding cassette (ABC) transporter.
[['20633576', 461, 498, 'Autosomal recessive Stargardt disease', 'D000080362', 'Disease', 2, '0#37'], ['20633576', 516, 533, 'macular dystrophy', 'D008268', 'Disease', 2, '55#72'], ['20633576', 579, 584, 'ABCA4', '24', 'Gene', 2, '118#123']]
Biochemical studies together with analysis of abca4 knockout mice and Stargardt patients have implicated ABCA4 as a lipid transporter that facilitates the removal of potentially toxic retinal compounds from photoreceptors following photoexcitation.
[['20633576', 688, 693, 'abca4', '24', 'Gene', 3, '46#51'], ['20633576', 712, 721, 'Stargardt', 'D000080362', 'Disease', 3, '70#79'], ['20633576', 747, 752, 'ABCA4', '24', 'Gene', 3, '105#110']]
An autosomal dominant form of Stargardt disease also known as Stargardt-like dystrophy is caused by mutations in a gene encoding ELOV

Testing for a gap junction-mediated bystander effect in retinitis pigmentosa: secondary cone death is not altered by deletion of connexin36 from cones.
[['23468924', 56, 76, 'retinitis pigmentosa', 'C538365', 'Disease', 0, '56#76'], ['23468924', 129, 139, 'connexin36', '57369', 'Gene', 0, '129#139']]
We tested this hypothesis in two different mouse models for retinal degeneration (rhodopsin knockout and rd1) by crossbreeding them with connexin36-deficient mice as connexin36 represents the gap junction protein on the cone side and lack thereof most likely disrupts rod-cone coupling.
[['23468924', 936, 956, 'retinal degeneration', 'D012162', 'Disease', 6, '60#80'], ['23468924', 981, 984, 'rd1', '5158', 'Gene', 6, '105#108'], ['23468924', 1013, 1023, 'connexin36', '57369', 'Gene', 6, '137#147'], ['23468924', 1042, 1052, 'connexin36', '57369', 'Gene', 6, '166#176']]
Using immunohistochemistry, we compared the progress of cone degeneration between connexin36-deficient mouse mutants and thei

Activin receptor-like kinase 1 (ALK1) is an endothelial-specific member of the TGF-beta/BMP receptor family that is inactivated in patients with hereditary hemorrhagic telangiectasia (HHT).
[['22421041', 76, 106, 'Activin receptor-like kinase 1', '94', 'Gene', 1, '0#30'], ['22421041', 108, 112, 'ALK1', '94', 'Gene', 1, '32#36'], ['22421041', 221, 258, 'hereditary hemorrhagic telangiectasia', 'D013683', 'Disease', 1, '145#182'], ['22421041', 260, 263, 'HHT', 'D013683', 'Disease', 1, '184#187']]
Blocking Alk1 signaling during postnatal development in mice leads to retinal hypervascularization and the appearance of arteriovenous malformations (AVMs).
[['22421041', 434, 438, 'Alk1', '94', 'Gene', 4, '9#13'], ['22421041', 546, 573, 'arteriovenous malformations', 'D001165', 'Disease', 4, '121#148']]
Taken together, these results uncover a direct link between ALK1 and Notch signaling during vascular morphogenesis that may be relevant to the pathogenesis of HHT vascular lesions.
[['22421041', 

Here we show that IL-10 sensing by innate immune cells, independent of its effects on T cells, was critical for regulating mucosal homeostasis.
[['24792912', 307, 312, 'IL-10', '3586', 'Gene', 2, '18#23'], ['24792912', 412, 431, 'mucosal homeostasis', 'D052016', 'Disease', 2, '123#142']]
Following wild-type (WT) CD4(+) T cell transfer, Rag2(-/-)Il10rb(-/-) mice developed severe colitis in association with profound defects in generation and function of Treg cells.
[['24792912', 482, 486, 'Rag2', '5897', 'Gene', 3, '49#53'], ['24792912', 491, 497, 'Il10rb', '3588', 'Gene', 3, '58#64'], ['24792912', 525, 532, 'colitis', 'D003092', 'Disease', 3, '92#99']]
Importantly, transfer of WT but not Il10rb(-/-) anti-inflammatory macrophages ameliorated colitis induction by WT CD4(+) T cells in Rag2(-/-)Il10rb(-/-) mice.
[['24792912', 824, 830, 'Il10rb', '3588', 'Gene', 5, '36#42'], ['24792912', 878, 885, 'colitis', 'D003092', 'Disease', 5, '90#97'], ['24792912', 920, 924, 'Rag2', '5897', 'Gene', 5,

...

Proteomics analysis of bladder cancer invasion: Targeting EIF3D for therapeutic intervention.
[['29050215', 23, 37, 'bladder cancer', 'D001749', 'Disease', 0, '23#37'], ['29050215', 58, 63, 'EIF3D', '8664', 'Gene', 0, '58#63']]
These included proteins previously associated with bladder cancer and also additional novel such as PGRMC1, FUCA1, BROX and PSMD12, which were further confirmed by immunohistochemistry.
[['29050215', 767, 781, 'bladder cancer', 'D001749', 'Disease', 5, '51#65'], ['29050215', 816, 822, 'PGRMC1', '10857', 'Gene', 5, '100#106'], ['29050215', 824, 829, 'FUCA1', '2517', 'Gene', 5, '108#113'], ['29050215', 831, 835, 'BROX', '148362', 'Gene', 5, '115#119'], ['29050215', 840, 846, 'PSMD12', '5718', 'Gene', 5, '124#130']]
Pathway and interactome analysis predicted strong activation in muscle invasive bladder cancer of pathways associated with protein synthesis e.g. eIF2 and mTOR signaling.
[['29050215', 966, 996, 'muscle invasive bladder cancer', 'D001749', 'Disease', 6,

Latrophilin-1/ADGRL1 is a model aGPCR that regulates synapse formation and embryogenesis, and its mutations are associated with cancer and attention-deficit/hyperactivity disorder.
[['30428326', 429, 442, 'Latrophilin-1', '22859', 'Gene', 3, '0#13'], ['30428326', 443, 449, 'ADGRL1', '22859', 'Gene', 3, '14#20'], ['30428326', 557, 563, 'cancer', 'D009369', 'Disease', 3, '128#134'], ['30428326', 568, 608, 'attention-deficit/hyperactivity disorder', 'D001289', 'Disease', 3, '139#179']]
To get insights in the physiological impact of point mutations that alter Lphn1 signaling on an entire organism, we selected the mutations that exhibited the most effect in our in vitro assay and mutated them in the Lphn homolog lat-1 of the nematode C. elegans: (1) V1094I (L790A in C. elegans), which severely increased basal activity and abolished the response of the receptor to the agonist peptide; importantly, this is a mutation associated with upper aerodigestive tract cancer and maps to TM7 at the NPxx

PFKFB2 Promoter Hypomethylation as Recurrence Predictive Marker in Well-Differentiated Thyroid Carcinomas.
[['30884810', 0, 6, 'PFKFB2', '5208', 'Gene', 0, '0#6'], ['30884810', 87, 105, 'Thyroid Carcinomas', 'D013964', 'Disease', 0, '87#105']]
BRAF and TERT mutations have been extensively related to prognosis in thyroid cancer.
[['30884810', 206, 210, 'BRAF', '673', 'Gene', 2, '0#4'], ['30884810', 215, 219, 'TERT', '7015', 'Gene', 2, '9#13'], ['30884810', 276, 290, 'thyroid cancer', 'D013964', 'Disease', 2, '70#84']]
Papillary (PTC) and follicular (FTC) thyroid carcinomas are the most prevalent thyroid tumors, designated as well-differentiated thyroid carcinomas (WDTC).
[['30884810', 1659, 1662, 'PTC', '8030', 'Gene', 13, '11#14'], ['30884810', 1777, 1795, 'thyroid carcinomas', 'D013964', 'Disease', 13, '129#147'], ['30884810', 1727, 1741, 'thyroid tumors', 'D013959', 'Disease', 13, '79#93'], ['30884810', 1685, 1703, 'thyroid carcinomas', 'D013964', 'Disease', 13, '37#55']]
Papillary t

Conclusion: Overexpression of pPLAIIIbeta retarded cell elongation in all the tested organs except seeds, which were longer and thicker than those of the controls.
[['32148415', 1736, 1747, 'pPLAIIIbeta', '824660', 'Gene', 14, '30#41'], ['32148415', 1748, 1756, 'retarded', 'D008607', 'Disease', 14, '42#50']]
Based on gene structure and amino acid sequence similarity, 10 members of the pPLA family have been classified into three groups: pPLAI, pPLAII (alpha, beta, gamma, delta, and epsilon), and pPLAIII (alpha, beta, gamma, and delta) in Arabidopsis . Both pPLAI and pPLAIIs have been shown to be involved in the response of plants to auxin signaling, pathogens, and phosphate deficiency.
[['32148415', 2576, 2667, 'pPLAII (alpha, beta, gamma, delta, and epsilon), and pPLAIII (alpha, beta, gamma, and delta', '824660', 'Gene', 18, '137#228'], ['32148415', 2801, 2821, 'phosphate deficiency', 'D007015', 'Disease', 18, '362#382']]
ABCB4 gene mutations cause a wide spectrum of liver diseases, in

Studies based on early-onset familial AD (eFAD) models have supported that synaptic transmission is depressed by beta-amyloid (Abeta) triggered mechanisms.
[['32362813', 537, 548, 'familial AD', 'D000544', 'Disease', 5, '29#40'], ['32362813', 635, 640, 'Abeta', '351', 'Gene', 5, '127#132']]
Since endocytic traffic is essential for synaptic function, we reviewed Abeta-dependent and independent mechanisms of the earliest synaptic dysfunction in AD.
[['32362813', 1025, 1030, 'Abeta', '351', 'Gene', 9, '72#77'], ['32362813', 1108, 1110, 'AD', 'D000544', 'Disease', 9, '155#157']]
We have focused on the role of intraneuronal and secreted Abeta oligomers, highlighting the dysfunction of endocytic trafficking as an Abeta-dependent mechanism of synapse dysfunction in AD.
[['32362813', 1170, 1175, 'Abeta', '351', 'Gene', 10, '58#63'], ['32362813', 1247, 1252, 'Abeta', '351', 'Gene', 10, '135#140'], ['32362813', 1276, 1295, 'synapse dysfunction', 'D009461', 'Disease', 10, '164#183'], ['32362813',

In [ ]:
# go to RENET code and get classification result

In [49]:
RENET_df = pd.read_csv('RENET/classification_result.txt', sep='\t', header=0)
# RENET_df = RENET_df.rename(columns={"predict": "is_renet"})

In [50]:
RENET_df = RENET_df[RENET_df['predict']!=0]
predicted_positive = RENET_df.drop_duplicates(['pmid', 'geneId', 'diseaseId'])

In [51]:
predicted_positive.to_csv('RENET/classification_result.csv', index=None)

In [ ]:

# add folloing code in exp_ft
# reading BeFree and DT-Miner
global BeFree_df, DTMiner_df, BioBERT_df, old_R_df

old_R_df = pd.read_csv('../benchmark/RENET/classification_result.csv', header=0)
old_R_df = old_R_df.rename(columns={"predict": "is_old"})



old_R_df = set_df_to_str(old_R_df)



# BeFree_df, DTMiner_df = None, None
def evaluate_rst_all_info(ori_df, ori_c, ann_df, is_freeze_c = False, b_ann_df = False):
    global BeFree_df, DTMiner_df, BioBERT_df, old_R_df
    
    if not is_freeze_c:
        global ReNet_df

    _rst = []
    _df = ori_df.copy()
    _c = ori_c
    if not is_freeze_c:
        _c = 'pred_tmp'
        _df.rename(columns={ori_c:'pred_tmp'}, inplace=True)

    # predicted GDA
    _pred_N = sum(_df[_c])
    _rst.append(_pred_N)

    # precision on ann & based #
    S1 = get_scores_a(_df, _c, ann_df, 'new_label')
    _precision1, _recall, N_ann = S1
    _rst.extend([_precision1])

    # merging renet|befree|dtminer result first
    if not is_freeze_c:
        merge_t_df = _df.merge(BeFree_df, on=['pmid', 'geneId','diseaseId'], how='outer')
    else:
        merge_t_df = ReNet_df.merge(BeFree_df, on=['pmid', 'geneId','diseaseId'], how='outer')
        
    merge_t_df = merge_t_df.merge(DTMiner_df, on=['pmid', 'geneId','diseaseId'], how='outer')
    merge_t_df = merge_t_df.merge(BioBERT_df, on=['pmid', 'geneId','diseaseId'], how='outer')
    merge_t_df = merge_t_df.merge(old_R_df, on=['pmid', 'geneId','diseaseId'], how='outer')
    merge_t_df = merge_t_df.fillna(0)
    ann_positive = ann_df.copy()
    
    df=merge_t_df
    F=[1,1,1,1]
    c=[_c,'is_dtminer','is_befree','is_biobert', 'is_old']
#     print(c)
    if is_freeze_c:
        c=['is_renet','is_dtminer','is_befree','is_biobert', 'is_old']
    
    # c[0], c[2] = c[2], c[0]
    mdl_positive = df[(df[c[0]]==F[0]) & ((df[c[1]]==F[1]) & (df[c[2]]==F[2]) & (df[c[3]]==F[3]))]
    
    if isinstance(b_ann_df, pd.DataFrame):
        mdl_positive = b_ann_df.merge(mdl_positive, on=['pmid', 'geneId','diseaseId'], how='outer')
        mdl_positive = mdl_positive[mdl_positive['is_ner_error'].isna()]
        del mdl_positive['is_ner_error']
        
    olp_positive = ann_positive.merge(mdl_positive, on=['pmid', 'geneId','diseaseId'], how='outer')
    
    ann_n, mdl_n, olp_n = len(ann_positive), len(mdl_positive), len(olp_positive)
#     print('---', ann_n, mdl_n, olp_n)
    
#     print('model confidence GDA {}, adding {} to TP, total TP {}'.format(mdl_n, olp_n-ann_n, olp_n))
#     print(olp_positive.head(3))
    
    olp_positive['ann_label'] = olp_positive['new_label']
    olp_positive['ann_label'] = olp_positive['ann_label'].fillna(0)
    olp_positive = olp_positive[['pmid', 'geneId','diseaseId', 'new_label', 'ann_label']]
    olp_positive['new_label'] = olp_positive['new_label'].fillna(1)
    olp_positive.shape
    targe_df = merge_t_df.merge(olp_positive, on=['pmid', 'geneId','diseaseId'], how='left')
    targe_df['new_label'] = targe_df['new_label'].fillna(0)

    tar_df = _df.copy()
    if isinstance(b_ann_df, pd.DataFrame):
        tar_df = tar_df.merge(b_ann_df, on=['pmid', 'geneId','diseaseId'], how='left')
        tar_df['pred'] = tar_df.apply(lambda x: 1 if x[_c] ==1 and pd.isnull(x['is_ner_error']) else 0, axis=1)
    else:
        tar_df['pred'] = _df.apply(lambda x: 1 if x[_c] ==1 else 0, axis=1)
    S2 = get_scores_a(tar_df, 'pred', targe_df, 'new_label')
    _precision2, _, N_p2 = S2
#     print(S2)
    _rst.extend([_precision2])



    _rst.append(_recall)
#     print(','.join(list(map(str, _rst))))
    return _rst

def evaluate_rst_all_info_err(ori_df, ori_c, ann_df, _ann_df, abs_s_df_ner, is_freeze_c = False):
    _rst1 = evaluate_rst_all_info(ori_df, ori_c, ann_df, is_freeze_c)
    
    _rst2 = evaluate_rst_all_info(ori_df, ori_c, _ann_df, is_freeze_c, abs_s_df_ner)
#     _rst = _rst1[0], _rst1[1], _rst2[1], _rst1[2], _rst2[2], _rst1[4], _rst2[4], _rst1[6], _rst2[6]
#     _rst = _rst1[0], _rst1[2], _rst2[2], _rst1[4], _rst2[4], _rst1[6], _rst2[6]
    _rst = _rst1[0], _rst1[2], _rst2[2], _rst1[3], _rst2[3]
#     _rst = _rst1, _rst2
    return _rst


_rst = evaluate_rst_all_info(old_R_df, 'is_old', abs_s_df, is_freeze_c=True)
print(','.join(list(map(str, _rst))))
